In [42]:
import pandas as pd
import pycountry
import numpy as np

In [43]:
plumes = pd.read_csv(
    r'C:\Users\rebek\Ember\plume_perc\Plumes_V2.csv', # New GEM plume dataset
    thousands=",",       # interpret commas as thousand separators inside numbers
    quotechar='"',       # respect quoted fields
    engine="python"      # more flexible parser for tricky CSVs
)

plumes = plumes[plumes['Type of Infrastructure'] == 'coal mine']

plumes = plumes[plumes["Instrument"].isin(["NASA-JPL EMIT", "EnMAP - DLR", "Sentinel-2 - ESA", "Tanager", "PRISMA - ASI"])]

mines_with_plumes = plumes['GEM Infrastructure Name (Nearby)']

mines_with_plumes

1522    Shenjiazhuang Coal Mine
1525                        NaN
1526          Kaiyuan Coal Mine
1578                        NaN
1579           Qiyuan Coal Mine
                 ...           
3468                        NaN
3469                        NaN
3470    Shanxi Dongda Coal Mine
3471      Lu'an Licun Coal Mine
3472                        NaN
Name: GEM Infrastructure Name (Nearby), Length: 1460, dtype: object

In [44]:
mines = pd.read_csv(
    r'C:\Users\rebek\Ember\Data Tool\cmm-data-tool\query_result_2025-12-01T11_49_18.284701141Z.csv', # Reads in data from gem_mines_raw from metabase
    thousands=",",       # interpret commas as thousand separators inside numbers
    quotechar='"',       # respect quoted fields
    engine="python"      # more flexible parser for tricky CSVs
)

mines["IS_LIGNITE"] = mines["COAL_TYPE"].isin(
    ["Lignite"]
)

mines = mines[mines['STATUS'] == 'Operating']

non_lignite_mines = mines[mines['IS_LIGNITE'] == False]['MINE_NAME']

mines = mines[mines['IS_LIGNITE'] == False]

In [45]:
china_prod = pd.read_csv(
    r'C:\Users\rebek\Ember\Data Tool\cmm-data-tool\data_historical\assets\production\Global-Coal-Mine-Tracker-September-2024-Supplement-v2_china.csv',
    encoding='latin1',      # or 'ISO-8859-1'
    thousands=',',
    quotechar='"',
    engine='python'
)

# Keep only rows where 'Mine Name' is in the list of mine names
china_prod_filtered = china_prod[china_prod['GEM Mine ID'].isin(mines['GEM_MINE_ID'])]
# china_prod_filtered = china_prod[china_prod['Mine Name'].isin(mines['MINE_NAME'])]

# Optional: reset index after dropping rows
china_prod_filtered = china_prod_filtered.reset_index(drop=True)

In [46]:
# Read the non-China CSV
non_china_prod = pd.read_csv(
    r'C:\Users\rebek\Ember\Data Tool\cmm-data-tool\data_historical\assets\production\Global-Coal-Mine-Tracker-September-2024-Supplement-v2_non_china.csv',
    encoding='latin1',
    thousands=',',
    quotechar='"',
    engine='python'
)

non_china_prod.rename(columns={'ï»¿GEM Mine ID': 'GEM Mine ID'}, inplace=True) 

# Original filter: GEM Mine ID is in mines['GEM_MINE_ID']
mask_in_mines = non_china_prod['GEM Mine ID'].isin(mines['GEM_MINE_ID'])

# Rows where Lignite (Manual) is False
mask_not_lignite = non_china_prod['Lignite (Manual)'] == False

# Combine both conditions with OR
non_china_prod_filtered = non_china_prod[mask_in_mines | mask_not_lignite]

# Reset index
non_china_prod_filtered = non_china_prod_filtered.reset_index(drop=True)

In [47]:
non_china_prod_aligned = non_china_prod_filtered[china_prod_filtered.columns.intersection(non_china_prod_filtered.columns)]

historical_prod = pd.concat([china_prod_filtered, non_china_prod_aligned], ignore_index=True)

historical_prod['non_lignite'] = (
    historical_prod['Mine Name']
        .str.upper()
        .str.strip()
        .str.replace(r'[^\w\s]', '', regex=True)
        .isin(
            non_lignite_mines
                .str.upper()
                .str.strip()
                .str.replace(r'[^\w\s]', '', regex=True)
        )
)

historical_prod['plume'] = (
    historical_prod['Mine Name']
        .str.upper()
        .str.strip()
        .str.replace(r'[^\w\s]', '', regex=True)
        .isin(
            mines_with_plumes
                .str.upper()
                .str.strip()
                .str.replace(r'[^\w\s]', '', regex=True)
        )
)

output_columns = [
    'Coal Output (Annual, Mt) 2023', 
    'Coal Output (Annual, Mt) 2022', 
    'Coal Output (Annual, Mt) 2021', 
    'Coal Output (Annual, Mt) 2020', 
    'Coal Output (Annual, Mt) 2019', 
    'Coal Output (Annual, Mt) 2018', 
    'Coal Output (Annual, Mt) 2017'
]


# get latest value to align with Ioannis's oil/gas method
historical_prod['Most Recent Coal Output (Annual, Mt)'] = historical_prod[output_columns].bfill(axis=1).iloc[:, 0]

# Step 0: Replace '-' with NaN in Most Recent Coal Output
historical_prod['Most Recent Coal Output (Annual, Mt)'] = historical_prod['Most Recent Coal Output (Annual, Mt)'].replace('-', np.nan)

# Step 1: Create a mapping from MINE_NAME to the last PRODUCTION__MTPA value
mine_prod_map = mines.groupby('MINE_NAME')['PRODUCTION__MTPA'].last()

# Step 2: Fill missing values in Most Recent Coal Output by looking up Mine Name
historical_prod['Most Recent Coal Output (Annual, Mt)'] = historical_prod['Most Recent Coal Output (Annual, Mt)'].fillna(
    historical_prod['Mine Name'].map(mine_prod_map)
)

historical_prod = historical_prod[historical_prod['non_lignite'] == True]

In [48]:
mines = historical_prod.copy()

mines["Most Recent Coal Output (Annual, Mt)"] = pd.to_numeric(
    mines["Most Recent Coal Output (Annual, Mt)"], errors="coerce"
)

mines = mines.dropna(subset=["Country", "Most Recent Coal Output (Annual, Mt)"])

total_production = (
    mines
    .groupby("Country", as_index=False)["Most Recent Coal Output (Annual, Mt)"]
    .sum()
    .rename(columns={"Most Recent Coal Output (Annual, Mt)": "total_production_mtpa"})
)

plumes = mines[mines['plume'] == True]

plume_production = (
    plumes
    .groupby("Country", as_index=False)["Most Recent Coal Output (Annual, Mt)"]
    .sum()
    .rename(columns={"Most Recent Coal Output (Annual, Mt)": "plume_production_mtpa"})
)

# mines['has_plume']

coverage = (
    total_production
    .merge(plume_production, on="Country", how="left")
    .fillna({"plume_production_mtpa": 0})
)

coverage["percent_covered"] = (
    coverage["plume_production_mtpa"]
    / coverage["total_production_mtpa"]
    * 100
)

coverage = coverage.sort_values("percent_covered", ascending=False)
coverage = coverage[coverage["percent_covered"] > 0]
coverage = coverage[["Country", "percent_covered"]]
coverage = coverage.rename(columns={"percent_covered": "% hard coal production with plume".upper()})
coverage = coverage.sort_values("% hard coal production with plume".upper(), ascending=True)

In [49]:
# Compute world totals
world_total_production = mines["Most Recent Coal Output (Annual, Mt)"].sum()
world_plume_production = plumes["Most Recent Coal Output (Annual, Mt)"].sum()

world_percent = (
    world_plume_production / world_total_production * 100
    if world_total_production > 0 else 0
)

# Create WORLD row
world_row = pd.DataFrame({
    "Country": ["World"],
    "% hard coal production with plume".upper(): [world_percent]
})

# Append to bottom
coverage = pd.concat([coverage, world_row], ignore_index=True)

coverage

,Country,% HARD COAL PRODUCTION WITH PLUME
0,South Africa,6.708513
1,United States,7.725122
2,China,9.744656
3,Australia,20.106274
4,Russia,22.876950
5,Mexico,25.641026
6,Canada,31.402439
7,Poland,31.548894
8,Kazakhstan,34.778451
9,Ukraine,61.163388


In [50]:
coverage.to_csv("plume_percentage.csv", index=False)